# Sample Deconvolution Run

- Run the following cells for deconvolution using Yada.
- Example files are provided in data folder.

In [ ]:
!rm -r Yada
!git clone https://github.com/zurkin1/Yada.git
!cd Yada

## Configure input files (sample files are provided).

In [1]:
mix = 'data/mix.csv' #This is the mixture file in the format: columns: mix1, mix2, ..., rows: HUGO gene names.

## Select deconvolution methods (or leave default values if using PBMC cell types).

In order to simplify usage we tried not to require different platform, normalizations and scale parameters. Any such configuration can be added to the method.

In [2]:
from yada import *

cells_pbmc14 = ['memory.B.cells', 'naive.B.cells', 'memory.CD4.T.cells', 'naive.CD4.T.cells', 'regulatory.T.cells', 'memory.CD8.T.cells', 'naive.CD8.T.cells', 'NK.cells', 'neutrophils', 'monocytes', 'myeloid.dendritic.cells', 'macrophages', 'fibroblasts', 'endothelial.cells']
methods = [
            [10, #Number of runs.
             dtw_deconv, #Method.
             cells_pbmc14,
             'pure', #Reference matrix name. Our sample mix data is RNA-seq TPM normalized so we used an appropriate matrix.
            ]
        ]

## Run deconvolution.

In [4]:
from IPython.display import FileLink

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
run_deconv(mix, methods)

Deconvolution, num_mixes: 50
dtw_deconv, pure
 90%

,memory.B.cells,naive.B.cells,memory.CD4.T.cells,naive.CD4.T.cells,regulatory.T.cells,memory.CD8.T.cells,naive.CD8.T.cells,NK.cells,neutrophils,monocytes,myeloid.dendritic.cells,macrophages,fibroblasts,endothelial.cells
0,0.014645,0.029600,0.040753,1.245364e-01,0.070046,0.050309,0.065481,0.035109,0.078720,0.199857,0.069827,0.049677,0.068406,0.113122
1,0.000943,0.004477,0.037852,9.280187e-02,0.138013,0.054389,0.103446,0.031936,0.042627,0.107105,0.031515,0.036851,0.186932,0.109763
2,0.016589,0.034318,0.035900,8.756383e-02,0.066957,0.065564,0.087097,0.039704,0.056998,0.113446,0.041018,0.036074,0.113470,0.127191
3,0.067338,0.191888,0.061778,1.847298e-01,0.073284,0.048012,0.119735,0.037089,0.018065,0.053196,0.036359,0.029779,0.055372,0.061880
4,0.022060,0.059784,0.076542,2.518227e-01,0.036417,0.041011,0.078280,0.027855,0.018132,0.086570,0.036948,0.036382,0.111450,0.103079
5,0.041694,0.121558,0.063134,1.151075e-01,0.273905,0.052382,0.053412,0.054251,0.035832,0.072380,0.042482,0.012518,0.042683,0.009477
6,0.003103,0.006875,0.046453,1.507965e-01,0.035721,0.020195,0.067734,0.009136,0.049495,0.152455,0.040788,0.029556,0.084602,0.182416
7,0.028490,0.080119,0.050487,1.248888e-01,0.095116,0.047937,0.123007,0.034422,0.006527,0.025301,0.025334,0.010399,0.078767,0.115311
8,0.078281,0.224126,0.074046,1.748207e-01,0.101915,0.041864,0.040446,0.036017,0.012045,0.051072,0.031635,0.009590,0.072783,0.070155
9,0.023215,0.058937,0.088971,2.536176e-01,0.122907,0.048835,0.146646,0.041068,0.006160,0.009198,0.006456,0.022082,0.084787,0.081213


In [5]:
from IPython.display import FileLink, FileLinks

FileLink('data/results.csv')

c:\Work\src\Yada\data\results.csv